In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


# Install TensorFlow

import tensorflow as tf
import tensorflow_hub as hub


## Data Load

Loading random 5 drugs. Potentially we ould do a set with all the data, or drugs targeted for a particular cancer.

In [3]:
wes_efgr = pd.read_csv("../data/wes_ic50_Erlotinib_binary_resp.tsv",sep = "\t", index_col="COSMIC_ID")
wes_pacl = pd.read_csv("../data/wes_ic50_Paclitaxel_binary_resp.tsv",sep = "\t", index_col="COSMIC_ID")
wes_suni = pd.read_csv("../data/wes_ic50_Sunitinib_binary_resp.tsv",sep = "\t", index_col="COSMIC_ID")
wes_sora = pd.read_csv("../data/wes_ic50_Sorafenib_binary_resp.tsv",sep = "\t", index_col="COSMIC_ID")
wes_rapa = pd.read_csv("../data/wes_ic50_Rapamycin_binary_resp.tsv",sep = "\t", index_col="COSMIC_ID")

In [4]:
wes_multi_drug = pd.concat([wes_efgr, wes_pacl, wes_suni, wes_sora,wes_rapa], axis=0)

In [5]:
wes_efgr.describe()

,Unnamed: 0,1,2,9,10,12,13,14,15,16,...,101927722,101928638,102724473,102724928,105375355,105378803,107403068,109731405,DRUG_ID,LN_IC50
count,371.000000,371.000000,371.000000,371.000000,371.000000,371.00000,371.000000,371.000000,371.000000,371.000000,...,371.0,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.000000,371.0,371.000000
mean,185.000000,0.024259,0.032345,0.008086,0.021563,0.03504,0.002695,0.008086,0.010782,0.048518,...,0.0,0.008086,0.005391,0.016173,0.002695,0.021563,0.013477,0.029650,1.0,2.464419
std,107.242715,0.154059,0.177154,0.089680,0.145449,0.18413,0.051917,0.089680,0.103413,0.215147,...,0.0,0.089680,0.073323,0.126309,0.051917,0.145449,0.115462,0.169848,0.0,1.160602
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-3.132525
25%,92.500000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.044453
50%,185.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.661822
75%,277.500000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,3.192688
max,370.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,...,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,5.244164


In [6]:
#single drug output
wes_efgr.head()

,Unnamed: 0,1,2,9,10,12,13,14,15,16,...,101928638,102724473,102724928,105375355,105378803,107403068,109731405,DRUG_ID,LN_IC50,BINARY_RESPONSE
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
907268,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.726708,R
907269,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.175124,R
907270,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.332820,R
907271,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.894932,R
907273,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.717920,R


In [7]:
wes_multi_drug.head()

,Unnamed: 0,1,2,9,10,12,13,14,15,16,...,101928638,102724473,102724928,105375355,105378803,107403068,109731405,DRUG_ID,LN_IC50,BINARY_RESPONSE
COSMIC_ID,,,,,,,,,,,,,,,,,,,,,
907268,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.726708,R
907269,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.175124,R
907270,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,3.332820,R
907271,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.894932,R
907273,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.717920,R


In [8]:
len(wes_multi_drug)

1940

## Prep Data
Convert label to int, drop float version of response

In [9]:
wes_multi_drug.loc[(wes_multi_drug.BINARY_RESPONSE == 'S'),'BINARY_RESPONSE'] = 1
wes_multi_drug.loc[(wes_multi_drug.BINARY_RESPONSE == 'R'),'BINARY_RESPONSE'] = 0

wes_multi_drug = wes_multi_drug.drop('LN_IC50', axis=1)

In [10]:
# split data
train_data_set, dev_data_set, train_labels, dev_labels = train_test_split(wes_multi_drug, wes_multi_drug['BINARY_RESPONSE'], test_size=0.20, random_state=0)


### Translate to TF Data

In [11]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data_set.values, train_labels.values))
train_dataset = train_dataset.shuffle(len(train_data_set)).batch(1)

In [12]:
test_dataset = tf.data.Dataset.from_tensor_slices((dev_data_set.values, dev_labels.values))
test_dataset = test_dataset.shuffle(len(dev_data_set)).batch(1)

## Build Simple Model

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(units=128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.fit(train_dataset, epochs=10, steps_per_epoch=150)

Train for 150 steps
Epoch 1/10
150/150 [==============================] - 1s 9ms/step - loss: 0.2726 - accuracy: 0.8933
Epoch 2/10
150/150 [==============================] - 1s 7ms/step - loss: 0.3196 - accuracy: 0.9067
Epoch 3/10
150/150 [==============================] - 1s 6ms/step - loss: 0.2641 - accuracy: 0.8733
Epoch 4/10
150/150 [==============================] - 1s 7ms/step - loss: 0.3654 - accuracy: 0.8800
Epoch 5/10
150/150 [==============================] - 1s 7ms/step - loss: 0.5404 - accuracy: 0.8733
Epoch 6/10
150/150 [==============================] - 1s 7ms/step - loss: 0.4118 - accuracy: 0.8800
Epoch 7/10
150/150 [==============================] - 1s 7ms/step - loss: 0.3240 - accuracy: 0.8667
Epoch 8/10
150/150 [==============================] - 1s 7ms/step - loss: 0.8161 - accuracy: 0.8133
Epoch 9/10
150/150 [==============================] - 1s 7ms/step - loss: 0.4681 - accuracy: 0.8400
Epoch 10/10
150/150 [==============================] - 1s 7ms/step - loss: 0.311

## Eval Model

In [16]:
model.evaluate(test_dataset, verbose=2, steps=len(dev_data_set))

388/388 - 1s - loss: 0.4063 - accuracy: 0.8634


[0.4062731577685037, 0.86340207]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  2353280   
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 2,354,570
Trainable params: 2,354,570
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.save('wes_multi_model')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: wes_multi_model/assets


## Load Saved Model As Layer

In [19]:
hub_url = "wes_multi_model/"
embed = hub.KerasLayer(hub_url, input_shape=(18384,), dtype=tf.float64, trainable=False)

In [20]:
wes_multi_base_model = tf.keras.Sequential([
    embed,
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

In [21]:
wes_multi_base_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [22]:
wes_multi_base_model.fit(train_dataset, epochs=10, steps_per_epoch=150)

Train for 150 steps
Epoch 1/10
150/150 [==============================] - 1s 6ms/step - loss: 0.5875 - accuracy: 0.8200
Epoch 2/10
150/150 [==============================] - 0s 2ms/step - loss: 0.4125 - accuracy: 0.8867
Epoch 3/10
150/150 [==============================] - 0s 2ms/step - loss: 0.3631 - accuracy: 0.8733
Epoch 4/10
150/150 [==============================] - 0s 2ms/step - loss: 0.3350 - accuracy: 0.8800
Epoch 5/10
150/150 [==============================] - 0s 2ms/step - loss: 0.3244 - accuracy: 0.8867
Epoch 6/10
150/150 [==============================] - 0s 2ms/step - loss: 0.2953 - accuracy: 0.8933
Epoch 7/10
150/150 [==============================] - 0s 2ms/step - loss: 0.2565 - accuracy: 0.9067
Epoch 8/10
150/150 [==============================] - 0s 2ms/step - loss: 0.2950 - accuracy: 0.8867
Epoch 9/10
150/150 [==============================] - 0s 2ms/step - loss: 0.2427 - accuracy: 0.9067
Epoch 10/10
150/150 [==============================] - 0s 2ms/step - loss: 0.264

In [23]:
wes_multi_base_model.evaluate(test_dataset, verbose=2, steps=len(dev_data_set))

388/388 - 1s - loss: 0.3252 - accuracy: 0.8918


[0.32518822767954203, 0.8917526]

In [24]:
wes_multi_base_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                2354570   
_________________________________________________________________
dense_2 (Dense)              (None, 16)                176       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 2,354,763
Trainable params: 193
Non-trainable params: 2,354,570
_________________________________________________________________
